In [16]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import math
import sys

query_txt = input('크롤링 키워드 >> ')
cnt = int(input('크롤링 횟수 입력 >> '))
page_cnt = math.ceil(cnt/10)

path = 'D:/SJY/python/notebook/data/'
ft_name = input('텍스트 파일 이름 >> ')
ft_name = path + ft_name + '.txt'
fc_name = input('csv 파일 이름 >> ')
fc_name = path + fc_name + '.csv'
fx_name = input('xlsx 파일 이름 >> ')
fx_name = path + fx_name + '.xlsx'

path = "D:/SJY/python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://korean.visitkorea.or.kr")

time.sleep(2) # 2초간 페이지를 기다림

try:
    driver.find_element_by_xpath('''//*[@id="chkForm01"]''').click()
except:
    print('코로나 창이 없습니다.')
    
# 검색창의 이름을 찾아서 검색어 입력 후 검색
element = driver.find_element_by_id('inp_search')
element.send_keys(query_txt)
element.send_keys("\n") # 엔터 기능- > 검색 결과페이지로 이동

time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print("\n")
print("="*70)
print("요청하신 검색 건수는 {} 건이었고" .format(cnt))
print("총 {} 페이지의 게시글 수집을 진행하겠습니다" .format(page_cnt))
print("="*70)
print("\n")

#학습 목표 2: 페이지 이동하기
#Step 5. 페이지 번호를 바꾸면서 실제 데이터 추출하기
no = 1            # 게시글 번호 카운트용 변수
no2 =[ ]          # 게시글 번호 저장용 리스트
contents2=[ ]     # 게시글 내용 저장용 리스트
tags2=[ ]         # 해쉬 태그 정보 저장용 리스트

s_time = time.time( )
for x in range(1,page_cnt+1): # 1페이지부터 반복 시작    
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    print("\n")    
    
    time.sleep(2)              #페이지가 로딩 될 때까지 2초 대기
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul', 'list_thumType type1')
    
    for i in content_list:
        # 게시글 번호 입력
        no2.append(no)
        print('번호 : ', no)
        
        try :
            contents = i.find('div','tit').get_text( )
        except AttributeError :
            contents = '태그정보가 없습니다'
            contents2.append(contents)
            print('내용:',contents.strip())   
        else :    
            contents2.append(contents)
            print('내용:',contents.strip())
        
        try:
            tags = i.find('p', 'tag_type').get_text()
        except AttributeError:
            tags = '태그정보가 없습니다'
            tags2.append(tags)
            print('태그:',tags.strip())
            print("\n")
        else :
            tags2.append(tags)
            print('태그:',tags.strip())
            print("\n")
            
        if no == cnt :
            break
        
        no += 1
        
    time.sleep(2)  # 페이지 변경 전 2초 대기 

    x += 1 #다음페이지를 클릭해야하기때문, x=1이면 2를 클릭해야함
    if x == page_cnt+1 :   
         break
 
    if x == 6 :
        driver.find_element_by_link_text("다음").click()
    else :
        driver.find_element_by_link_text("{}".format(x)).click()  
        
#Step 6. 출력 결과를 표(데이터 프레임) 형태로 만들기
korea = pd.DataFrame()
korea['번호']=no2
korea['내용']=contents2
korea['태그']=tags2

# 추출 결과를 txt 파일로 저장하기
ft = open(ft_name, 'a',encoding='UTF-8')
ft.write(str(no2))
ft.write(str(contents2))
ft.write(str(tags2))
ft.close( )

# 추출 결과를 csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig",index=False)
 
# 추출 결과를 엑셀 형태로 저장하기
korea.to_excel(fx_name , index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ft_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)
 
driver.close( )

크롤링 키워드 >> 제주도 여행
크롤링 횟수 입력 >> 30
텍스트 파일 이름 >> t1
csv 파일 이름 >> t1
xlsx 파일 이름 >> t1


요청하신 검색 건수는 30 건이었고
총 3 페이지의 게시글 수집을 진행하겠습니다


1 페이지 내용 수집 시작합니다 =======================


번호 :  1
내용: 엄마 아빠에겐 쉼표, 아이에겐 느낌표! 아이와 떠나는 제주 여행
태그: #영유아추천코스#제주가볼만한곳#에코랜드#아쿠아플라넷제주#세계자동차제주박물관#헬로키티아일랜드#이니스프리제주하우스#한담해안산책로#테지움#제주1박2일코스#제주2박3일코스#제주3박4일코스#무장애여행지#체험여행#화산송이#아이와함께#아이와가볼만한곳


번호 :  2
내용: [국내여행 버킷리스트 1탄] '나도 제주 해녀가 되고 싶어요', 제주 하도어촌체험마을 해녀체험
태그: #제주도가볼만한곳#이색체험#섬여행#친구와함께#수상레포츠#바다여행#레포츠여행#하도어촌체험마을#해녀체험


번호 :  3
내용: 가족 단위 추천 느린 여행지 2탄 - 제주시 동부
태그: #비짓제주#제주가볼만한곳#제주_여행법#가족여행#가족과함께#제주절물자연휴양림#제주돌문화공원#선흘분교#아부오름#제주_가볼만한곳


번호 :  4
내용: 맛있는 음식에 여행의 기쁨은 두 배! <제주 식도락 여행>
태그: #비짓제주#제주여행#제주가볼만한곳#제주맛집#오쿠다#만민식당#알로하도#방긋스낵#곰막#실패없는_제주의맛#맛따라가기#제주여행법#제주_여행법


번호 :  5
내용: 태그정보가 없습니다
태그: 태그정보가 없습니다


번호 :  6
내용: [국내 트레킹 추천] 제주 올레길 걷기 여행, 준비부터 코스 선택까지 꿀팁 총정리!
태그: #제주올레길#제주올레걷기축제#제주올레#제주올레여행자센터#트레킹코스#트레킹#10월가볼만한곳#토닥토닥_걷기길


번호 :  7
내용: 한국관광공사 추천 2월 걷기여행길, 이른 봄을 맞이하는 제주도 걷기길
태그: #걷기여행길#2월걷기여행길#제주도#제주도걷기길#트레킹코스


번호 :  8
내용: 소소한 

In [17]:
korea

,번호,내용,태그
0,1,"엄마 아빠에겐 쉼표, 아이에겐 느낌표! 아이와 떠나는 제주 여행",#영유아추천코스#제주가볼만한곳#에코랜드#아쿠아플라넷제주#세계자동차제주박물관#헬로키티...
1,2,"[국내여행 버킷리스트 1탄] '나도 제주 해녀가 되고 싶어요', 제주 하도어촌체험...",#제주도가볼만한곳#이색체험#섬여행#친구와함께#수상레포츠#바다여행#레포츠여행#하도어촌...
2,3,가족 단위 추천 느린 여행지 2탄 - 제주시 동부,#비짓제주#제주가볼만한곳#제주_여행법#가족여행#가족과함께#제주절물자연휴양림#제주돌문...
3,4,맛있는 음식에 여행의 기쁨은 두 배! <제주 식도락 여행>,#비짓제주#제주여행#제주가볼만한곳#제주맛집#오쿠다#만민식당#알로하도#방긋스낵#곰막#...
4,5,태그정보가 없습니다,태그정보가 없습니다
5,6,"[국내 트레킹 추천] 제주 올레길 걷기 여행, 준비부터 코스 선택까지 꿀팁 총정리!",#제주올레길#제주올레걷기축제#제주올레#제주올레여행자센터#트레킹코스#트레킹#10월가볼...
6,7,"한국관광공사 추천 2월 걷기여행길, 이른 봄을 맞이하는 제주도 걷기길",#걷기여행길#2월걷기여행길#제주도#제주도걷기길#트레킹코스
7,8,소소한 '제주여행' 꿀팁! 〈겨울편〉,#제주도#제주도여행코스#라프별빛축제#새별오름#카멜리아힐#향파두리유적지#제주가볼만한곳...
8,9,반려견과 떠나는 여행 <제주 반려견 숙소&카페>,#비짓제주#제주여행#제주가볼만한곳#멍멍플레이스#브레이브독스#카페비글#카페3535.1...
9,10,가족 단위 추천 느린 여행지 1탄 - 제주시 서부,#비짓제주#제주여행#제주가볼만한곳#동명리#명월리#월령리#더럭초등학교#가족여행#가족과...
